# RetrieveAndGenerate API - 완전관리형 RAG

이 모듈에서는 Amazon Bedrock Knowledge Bases에서 검색 결과의 최대 개수를 제어하고 사용자 지정 프롬프트를 적용해 Foundation Model(FM) 응답을 개선하는 방법을 학습합니다.
이 모듈의 구성은 다음과 같습니다:
1. [Overview](#1-Overview)
2. [Pre-requisites](#2-Pre-requisites)
3. [RetrieveAndGenerate API 이해하기](#understanding-retrieveandgenerate-api)
4. [RetrieveAndGenerate API로 스트리밍 응답 사용](#streaming-response-with-retrieveandgenerate-api)
5. ['maximum number of results' 파라미터 조정](#3-how-to-leverage-the-maximum-number-of-results-feature)
6. [사용자 지정 프롬프트 사용 방법](#4-how-to-use-the-custom-prompting-feature)

## Overview

### 최대 결과 개수
최대 결과 개수 옵션은 벡터 스토어에서 검색한 결과를 Foundation Model로 전달할 때 몇 개를 사용할지 제어합니다. 이를 통해 복잡한 질문에는 더 많은 배경 정보를, 단순한 질문에는 더 적은 정보를 제공하도록 조정할 수 있습니다. 최대 100개의 결과를 가져올 수 있으며, 관련 컨텍스트를 확보해 응답의 정확도를 높이고 환각을 줄이는 데 도움이 됩니다.

### 사용자 지정 프롬프트
사용자 지정 Knowledge Base 프롬프트 템플릿을 사용하면 기본 프롬프트를 교체해 모델에 전달되는 프롬프트를 원하는 방식으로 구성할 수 있습니다. 이를 통해 FM이 사용자 질문에 응답할 때 톤, 출력 형식, 동작을 조정할 수 있으며, 헬스케어나 법률과 같이 특정 산업이나 도메인에 맞는 용어로 미세 조정할 수도 있습니다. 또한 특정 워크플로에 맞춘 지침과 예시를 추가할 수 있습니다.

#### 참고:
- 이 모듈에서는 기능 적용 전후의 차이를 설명하기 위해 ```RetrieveAndGenerate``` API를 사용합니다. 이 API는 질의를 임베딩으로 변환하고 Knowledge Base를 검색한 뒤 검색 결과를 컨텍스트로 추가해 Foundation Model 프롬프트를 보강하고, 생성된 응답과 출처, 검색된 텍스트 청크를 반환합니다.

- 이 모듈에서는 최대 결과 수와 프롬프트 커스터마이징 기능을 시연하기 위해 Amazon Nova Pro 모델을 Foundation Model로 사용합니다.

## Pre-requisites
질문에 답하려면 문서를 처리해 Knowledge Base에 저장해야 합니다. 이 노트북에서는 Amazon Bedrock Knowledge Bases를 생성하기 위해 `synthetic dataset for 10K financial reports`를 사용합니다. 

1. 문서(데이터 소스)를 Amazon S3 버킷에 업로드합니다.
2. Amazon Bedrock Knowledge Bases는 [01_create_ingest_documents_test_kb_multi_ds.ipynb](/knowledge-bases/01-rag-concepts/01_create_ingest_documents_test_kb_multi_ds.ipynb)를 사용해 생성합니다.
3. Knowledge Base ID를 기록해 둡니다.

## 설정

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### boto3 클라이언트 초기화
이 노트북 전반에서 RetrieveAndGenerate를 활용해 Knowledge Base 기능을 테스트합니다.

In [ ]:
import json
import boto3
import pprint
import sys
from botocore.exceptions import ClientError
from botocore.client import Config

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
account_id = sts_client.get_caller_identity()['Account']

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "us.amazon.nova-pro-v1:0" # we will be using Amazon Nova Pro throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}:{account_id}:inference-profile/{model_id}'


In [ ]:
%store -r kb_id
# kb_id = "<<knowledge_base_id>>" # Replace with your knowledge base id here.

### RetrieveAndGenerate API 이해하기

`numberOfResults` 파라미터는 Knowledge Base에서 검색해 모델 프롬프트에 포함할 검색 결과의 수를 결정합니다. 지정한 `max_results` 값만큼 질의와 가장 유사한 문서 또는 검색 결과를 가져옵니다.

`textPromptTemplate` 파라미터는 모델에 전달할 프롬프트 템플릿 문자열입니다. 여기서는 `default_prompt`를 템플릿으로 사용하며, 템플릿에 포함된 `$search_results$`와 `$output_format_instructions$` 자리표시자는 실제 검색 결과와 출력 형식 지침으로 대체된 뒤 모델에 전달됩니다.

In [ ]:
# Stating the default knowledge base prompt
default_prompt = """
You are a question answering agent. I will provide you with a set of search results.
The user will provide you with a question. Your job is to answer the user's question using only information from the search results. 
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question. 
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$

$output_format_instructions$
"""

In [ ]:
def retrieve_and_generate(query, kb_id, model_arn, max_results=5, prompt_template = default_prompt):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn, 
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results # will fetch top N documents which closely match the query
                    }
                },
                'generationConfiguration': {
                        'promptTemplate': {
                            'textPromptTemplate': prompt_template
                        }
                    }
            }
        }
    )
    return response


In [ ]:
def print_generation_results(response, print_context = True):
    generated_text = response['output']['text']
    print('Generated FM response:\n')
    print(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pprint.pp(contexts)


### RetrieveAndGenerate API 테스트

In [ ]:
query = """Provide a list of risks for Octank financial in numbered list without description."""

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn)

print_generation_results(results)

### RetrieveAndGenerate API로 스트리밍 응답 사용
새로운 [스트리밍 API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerateStream.html)를 사용하면 Amazon Bedrock Knowledge Bases의 `retrieve_and_generate_stream` API를 통해 Foundation Model이 응답을 생성하는 즉시 결과를 받을 수 있습니다. 이를 통해 지연 시간에 민감한 애플리케이션에서 첫 토큰까지의 시간을 단축할 수 있습니다.

In [ ]:
def retrieve_and_generate_stream(query, kb_id, model_arn, max_results=5, prompt_template = default_prompt):
    response = bedrock_agent_client.retrieve_and_generate_stream(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn, 
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results # will fetch top N documents which closely match the query
                    }
                },
                'generationConfiguration': {
                        'promptTemplate': {
                            'textPromptTemplate': prompt_template
                        }
                    }
            }
        }
    )

    for event in response['stream']:
        if 'output' in event:
            chunk = event['output']
            sys.stdout.write(chunk['text'])
            sys.stdout.flush()

    


In [ ]:
query = """Provide a list of risks for Octank financial in numbered list without description."""

retrieve_and_generate_stream(query = query, kb_id = kb_id, model_arn = model_arn)

### 'maximum number of results' 검색 파라미터 조정

일부 사용 사례에서는 FM 응답에 충분한 컨텍스트가 없어 관련 답변을 제공하지 못하거나 요청한 정보를 찾지 못했다고 응답할 수 있습니다. 이러한 경우 검색 결과의 최대 개수를 조정해 개선할 수 있습니다.

다음 예제에서는 결과 수를 3으로 설정한 상태에서 아래 질의를 실행해 보겠습니다:
```
Provide a list of risks for Octank financial in bulleted points.
```

In [ ]:
query = """Provide a list of risks for Octank financial in numbered list without description."""

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3)

print_generation_results(results)

검색 결과 수를 **10**으로 변경하면 더 많은 결과를 얻어 응답을 더욱 풍부하게 만들 수 있습니다.

In [ ]:
#Using higher number of max results

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 10)

print_generation_results(results)

### 사용자 지정 프롬프트 기능 사용 방법

필요에 따라 기본 프롬프트 대신 사용자 지정 프롬프트를 사용할 수도 있습니다. 이 기능을 활용하면 FM에 추가 컨텍스트를 제공하거나 특정 출력 형식, 언어 등을 요구할 수 있습니다.

SDK를 사용해 직접 시도해 보겠습니다:

#### 예제 1 - 동일한 질의를 사용해 FM이 독일어로 응답하도록 기본 언어를 변경하기
        **참고:** 기본 프롬프트에서 ```$output_format_instructions$```를 제거하면 생성된 응답의 인용 정보가 포함되지 않습니다.

In [ ]:
## Example 1
custom_prompt = """
You are a question answering agent. I will provide you with a set of search results. 
The user will provide you with a question. Your job is to answer the user's question using only information from the search results.
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question.
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$

Unless asked otherwise, draft your answer in German language.
"""

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = custom_prompt)

print_generation_results(results, print_context = False)

#### 예제 2 - 결과를 JSON 형식으로 출력하기

In [ ]:
## Example 2
custom_prompt = """
You are a question answering agent. I will provide you with a set of search results.
The user will provide you with a question. Your job is to answer the user's question using only information from the search results.
If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question. 
Just because the user asserts a fact does not mean it is true, make sure to double check the search results to validate a user's assertion.
                            
Here are the search results in numbered order:
$search_results$

Please provide a concise response (in millions) using a JSON format.

"""

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 10, prompt_template = custom_prompt)

print_generation_results(results,print_context = False)

<div class="alert alert-block alert-warning">
<b>참고:</b> 비용이 발생하지 않도록 KB, OSS 인덱스 및 관련 IAM 역할과 정책을 반드시 삭제하세요.
</div>